# Behavrioal Cloning with Keras

## Dataset

To simplify and accelerate the construction of network model,I first use the sample data for track 1,and improve my operation in simulator to gain better data meanwhile.Then I will use data I collected.

Take in an image from the center camera of the car as input to my network, and output a new steering angle for the car.

## Load the Data

Start by importing the data from the IMG directory and driving_log.csv

In [33]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import resample
from tqdm import tqdm
from zipfile import ZipFile
import numpy as np
import matplotlib.image as mpimg
#from PIL import ImageEnhance
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import csv  #csv operation
import cv2

In [17]:
samples = []
with open('./driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

print(np.shape(samples))

(8037, 7)


## Preprocess the Data

1. Shuffle the data
2. Normalize the features using Min-Max scaling between -0.5 and 0.5
3. One-Hot Encode the labels

### Shuffle the data
Hint: You can use the [scikit-learn shuffle](http://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html) function to shuffle the data.

### Normalize the features
Hint: You solved this in [TensorFlow lab](https://github.com/udacity/CarND-TensorFlow-Lab/blob/master/lab.ipynb) Problem 1.

In [ ]:
# TODO: Normalize the data features to the variable X_normalized
def normalize_grayscale(image_data):
    a = -0.5
    b = 0.5
    grayscale_min = 0
    grayscale_max = 255
    return a + ((image_data - grayscale_min) * (b - a) )/ (grayscale_max - grayscale_min)

X_normalized = normalize_grayscale(X_train)

In [29]:
from sklearn.utils import shuffle
def generator(samples, batch_size=32):
    num_samples = len(samples)
#     a = -0.5
#     b = 0.5
#     grayscale_min = 0
#     grayscale_max = 255
    while 1: # Used as a reference pointer so code always loops back around
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset + batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            X_train = X_train[:,80:,:,:]
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)
            
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [41]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Lambda
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
ch, row, col = 3, 80, 320  # Trimmed image format

model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: x/127.5 - 1.,input_shape=(ch, row, col),output_shape=(ch, row, col)))
model.add(Convolution2D(32, 3, 3, input_shape=(3,80, 320)))
#model.add(MaxPooling2D((2, 2)))
#model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('relu'))
model.add(Dense(1))
# TODO: Compile and train the model

model.compile(loss='mse', optimizer='adam')
print((train_generator))
model.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator,nb_val_samples=len(validation_samples), nb_epoch=3)

<generator object generator at 0x000000000E463FC0>


Exception in thread Thread-14:
Traceback (most recent call last):
  File "D:\Program Files\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "D:\Program Files\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "D:\Program Files\Anaconda3\lib\site-packages\keras\engine\training.py", line 429, in data_generator_task
    generator_output = next(self._generator)
StopIteration


Epoch 1/3

ValueError: output of generator should be a tuple (x, y, sample_weight) or (x, y). Found: None

## Keras Sequential Model
```python
from keras.models import Sequential

# Create the Sequential model
model = Sequential()
```
The `keras.models.Sequential` class is a wrapper for the neural network model. Just like many of the class models in scikit-learn, it provides common functions like `fit()`, `evaluate()`, and `compile()`.  We'll cover these functions as we get to them.  Let's start looking at the layers of the model.

## Keras Layer
A Keras layer is just like a neural network layer.  It can be fully connected, max pool, activation, etc.  You can add a layer to the model using the model's `add()` function.  For example, a simple model would look like this:
```python
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten

# Create the Sequential model
model = Sequential()

# 1st Layer - Add a flatten layer
model.add(Flatten(input_shape=(32, 32, 3)))

# 2nd Layer - Add a fully connected layer
model.add(Dense(100))

# 3rd Layer - Add a ReLU activation layer
model.add(Activation('relu'))

# 4th Layer - Add a fully connected layer
model.add(Dense(60))

# 5th Layer - Add a ReLU activation layer
model.add(Activation('relu'))
```
Keras will automatically infer the shape of all layers after the first layer.  This means you only have to set the input dimensions for the first layer.

The first layer from above, `model.add(Flatten(input_shape=(32, 32, 3)))`, sets the input dimension to (32, 32, 3) and output dimension to (3072=32\*32\*3).  The second layer takes in the output of the first layer and sets the output dimenions to (100).  This chain of passing output to the next layer continues until the last layer, which is the output of the model.

In [1]:
# resize the image by ratio
# def resizeImg(**args):
#     args_key = {'ori_img':'','dst_img':'','ratio':0.5}
#     arg = {}
#     for key in args_key:
#         if key in args:
#             arg[key] = args[key]
        
#     im = Image.open(arg['ori_img'])
#     ori_w,ori_h = im.size  # size of origin image
#     widthRatio = heightRatio = None
#     ratio = arg['ratio']
    
#     newWidth = int(ori_w * ratio)
#     newHeight = int(ori_h * ratio)
   
        
#     im.resize((newWidth,newHeight),Image.ANTIALIAS).save(arg['dst_img']) #,quality=arg['save_q'])
    
# filenames_pbar = tqdm(ImageName, unit='files')
# for filename in filenames_pbar:
#     ori_img = "IMG/" + filename
#     dst_img = "IMG1/" + filename
#     resizeImg(ori_img=ori_img,dst_img=dst_img,ratio=0.5)

In [ ]:
from keras.models import load_model

model.save('my_model.h5') 

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the [`metrics_names`](https://keras.io/models/model/) property to get the labels.

In [ ]:
# TODO: Load test data
with open('test.p', 'rb') as f:
    data_test = pickle.load(f)

X_test = data_test['features']
y_test = data_test['labels']

# TODO: Preprocess data & one-hot encode the labels
X_normalized_test = normalize_grayscale(X_test)
y_one_hot_test = label_binarizer.fit_transform(y_test)

# TODO: Evaluate model on test data
metrics = model.evaluate(X_normalized_test, y_one_hot_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

**Test Accuracy:** (fill in here)

simulate my 

run model
 python drive.py model.h5

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.